# Useful functions for CV applications
This notebook has a number of useful functions to refer to for doing various CV tasks

### Create Path Dictionary
This function will take in a folder path along with child folders to read in image data. Valid extensions can be set so only those images meeting those requirements will be read

In [ ]:
import os

# Examples of variables to pass to the following function
folder_path = '<your path>'
folder_list = ['A', 'B', 'C', 'D', 'E']
valid_extensions = ['.jpg', '.png']

# Requires all the paths and validity checks as input
# outputs a dictionary with the key value being the folder name
# and a list of [list of files]
def createPathDict (folder_path, folder_list, valid_extensions):
    
    pathDict = {}
    
    for name in folder_list:
        # crazy single liner
        # adds a list of class file names to the dictionary of paths for that class, only if it is a file
        # and it has an extensions matching the valid extensions list, ruling out non image types
        class_list = [folder_path + '/' + name + '/' + f for f in os.listdir(folder_path + '/' + name) if os.path.isfile(os.path.join(folder_path + '/' + name, f)) for i in valid_extensions if f.endswith(i) > 0]
        pathDict[name] = class_list
    
    return pathDict

### Face Detection function 
Quick function to use in conjunction with the previously mentioned path dictionary. Quickly identifies faces found in the images and returns a dictionary with folder: imagename: [image, faces]

In [2]:
import os
import cv2

OPENCV_PATH = "/usr/local/lib/python3.6/dist-packages/cv2/data"
faceXML = os.path.join(OPENCV_PATH,'haarcascade_frontalface_default.xml')
faceCascade = cv2.CascadeClassifier(faceXML)

# cycles through a dictionary of paths as found in the functions previous
# outputs a dictionary in the form of {folder: {imagename: [image, face rectangle]}}
def processImagesFromPathDict(faceCascade, pathDict, scaleFactor=1.1, minNeighbors=8):
    imageDictionary = {}
    for key in pathDict:
        imageDictionary[key] = {}
        for value in pathDict[key]:
            imageFile = value
            filename = value.split('/')[-1]
            # Reads image into RGB for histogram comparison
            img = cv2.imread(imageFile)
            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            faces = faceCascade.detectMultiScale(img_rgb, scaleFactor=scaleFactor, minNeighbors=minNeighbors)
            imageDictionary[key][filename] = (img_rgb, faces)  
    return imageDictionary

### Slice images of faces
Used in conjunction with the above functions to slice the image array into just the face data for processing. Drastically reduces compute complexity by only processing a smaller area.

In [3]:
# pretty agressive function name
def sliceFace(imageDict):
    faceDict = {}
    for foldername in imageDict:
        faceDict[foldername] = {}
        for imageName in imageDict[foldername]:
            faceDict[foldername][imageName] = []
            faceData = imageDict[foldername][imageName][1]
            for (x,y,w,h) in faceData:
                rectangle = x,y,w,h
                faceimg = imageDict[foldername][imageName][0][y:y+h, x:x+w]
                faceDict[foldername][imageName].append((faceimg, rectangle))
   
    return faceDict